# **LoopsResultsAnalysis**

## **1. Importing and First Proccesing**

In [9]:
from datetime import datetime as dt
import pandas as pd
from scipy import stats
import plotly.express as px
from DataCleaning import *
from ProcessingConfig import *

pd.options.display.max_columns = 50

In [3]:
raw_data = pd.read_excel(cleaning_config['raw_data_path'])
print(f'original shape: {raw_data.shape}')
outliers_threshold = cleaning_config['filter_threshold']
print(f"threshold for outliers detection: {outliers_threshold}")

drop_columns(raw_data, cleaning_config['unnecessary_columns'])
convert_types(raw_data, cleaning_config['type_conversions'])
raw_data = filter_slow_subjects(raw_data, outliers_threshold)
raw_data = filter_bad_subjects(raw_data, outliers_threshold)
raw_data = drop_first_loop(raw_data)
raw_data = only_first_line(raw_data)
raw_data = filter_bad_trials(raw_data, threshold=0.5)
raw_data = filter_slow_steps(raw_data, outliers_threshold)

path = cleaning_config['results_path'] + f'_{dt.now().strftime("%d.%m.%Y_%H-%M")}.xlsx'
raw_data.to_excel(path)

print(f'final shape: {raw_data.shape}')

original shape: (13294, 32)
threshold for outliers detection: 2.25
filter_slow_subjects: No slow subjects detected.
filter_bad_subjects: No bad subjects detected (in terms of low success rate).
drop_first_loop: 1630 rows were filtered out.
only_first_line: 9227 rows were filtered out.
filter_bad_trials: No bad trials detected (in terms of low success rate).
filter_slow_steps: 77 slow steps were filtered out.
Here is a summary of slow steps rate per subjects: 
              slow steps rate (%)
subject                     
104A                    0.64
102A                    1.96
106A                    1.96
106B                    1.96
103A                    2.38
102B                    2.58
101B                    3.29
105A                    3.42
109B                    3.87
108A                    3.90
107B                    4.00
104B                    5.11
108B                    5.48
109A                    5.48
103B                    7.09
final shape: (2360, 25)


In [4]:
n_subjects = raw_data['subject'].nunique()
print(f'number of subjects: {n_subjects}')

n_sessions = raw_data[['subject', 'trial_set']].drop_duplicates().shape[0]
print(f'number of sessions: {n_sessions}')

number of subjects: 15
number of sessions: 29


In [17]:
total_times = raw_data[['subject', 'trial_set', 'rt']].groupby(['subject', 'trial_set'])['rt'].sum() / (1000)

times_hist = px.histogram(total_times, nbins=10, x='rt'
                   , title='Total Time per Session', labels={'rt':'response time sum'})
times_hist.show()

## **2. Priming Effect Testing**

In [5]:
alpha = 0.05

# enveloping t_test and f_test functions

def f_test(smp1, smp2, alpha):
    f_score = smp1.std() / smp2.std()
    df1, df2 = smp1.size - 1, smp2.size - 1
    return stats.f.cdf(f_score, df1, df2) >= alpha

def t_test_ind(smp1, smp2, alpha, alternative='two-sided'):
    equal_var = f_test(smp1, smp2, alpha)
    t_score, p_value = stats.ttest_ind(smp1, smp2, alternative=alternative, equal_var=equal_var)
    
    print(f'p_value: {p_value}')
    if p_value <= alpha:
        print(f'There is a significant difference between the samples! ({(1-alpha)*100}%).')
    else:
        print(f'It is not possible to determine whether there is an effect ({(1-alpha)*100}%).')
    return t_score, p_value

In [6]:
# mean response time and success rate, group by loop type and loop switching.
switching_diff = pd.DataFrame()

switching_diff['mean_response_time'] = raw_data[raw_data['correct']].groupby(['loop_type', 'loop_type_switch'])['rt'].mean()
switching_diff['mean_success_rate'] = raw_data.groupby(['loop_type', 'loop_type_switch'])['correct'].mean()

switching_diff

mean_response_time  mean_success_rate
loop_type loop_type_switch                                       
for       False                    3254.087179           0.976628
          True                     3249.750433           0.976311
while     False                    3536.781362           0.953846
          True                     3670.382979           0.964103

In [7]:
# checking for priming effect on 'for' loops
same = raw_data.loc[raw_data['correct'] & ( ~ raw_data['loop_type_switch']) & (raw_data['loop_type'] == 'for'), 'rt']
different = raw_data.loc[raw_data['correct'] & (raw_data['loop_type_switch']) & (raw_data['loop_type'] == 'for'), 'rt']

t_score, p_value = t_test_ind(different, same, alpha=alpha, alternative='greater')

p_value: 0.5169306161550555
It is not possible to determine whether there is an effect (95.0%).


In [22]:
# checking for priming effect on 'while' loops
same = raw_data.loc[raw_data['correct'] & ( ~ raw_data['loop_type_switch']) & (raw_data['loop_type'] == 'while'), 'rt']
different = raw_data.loc[raw_data['correct'] & (raw_data['loop_type_switch']) & (raw_data['loop_type'] == 'while'), 'rt']

t_score, p_value = t_test_ind(different, same, alpha=alpha, alternative='greater')

p_value: 0.12093149698869998
It is not possible to determine whether there is an effect (95.0%).
